### 대화 메모리 관리
- 대화 메모리(MessageHistory)는 AI Agent가 이전 대화 내용을 메시지 단위로 저장하고,
이를 바탕으로 **맥락(Context)**을 유지하여 자연스러운 대화를 이어가기 위한 구조이다.
- 단순한 “대화 이력 저장”을 넘어서, 대화 요약(Summarization)과 의도 추적(Context Tracking) 기능을 포함한다.

In [1]:
import os
from dotenv import load_dotenv

# .env 파일의 내용 불러오기
load_dotenv("C:/env/.env")

True

### [1] ConversationBufferMemory: 구버전  
(LangChain ≥ 0.2.7 신버전에서 Deprecated, 현재 버전에서 실행 오류)

In [2]:
# from langchain_openai import ChatOpenAI
# from langchain.chains import ConversationChain
# from langchain.memory import ConversationBufferMemory

# # 1) LLM 모델 정의
# llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)

# # 2) 메모리 객체 생성
# # return_messages=True → 각 대화를 메시지 객체로 반환
# memory = ConversationBufferMemory(return_messages=True)

# # 3) 대화 체인(ConversationChain) 구성
# conversation = ConversationChain(
#     llm=llm,
#     memory=memory,
#     verbose=True   # 내부 동작 로그 출력
# )

# # 4) 대화 실행
# print("=== 대화 시작 ===")
# response1 = conversation.predict(input="안녕? 나는 홍길동이야.")
# print("🤖:", response1)

# response2 = conversation.predict(input="내 이름이 뭐지?")
# print("🤖:", response2)

# response3 = conversation.predict(input="좋아하는 음식은 뭐야?")
# print("🤖:", response3)

# # 5) 메모리 내용 확인
# print("\n=== 현재 메모리 저장 내용 ===")
# print(memory.buffer)


### [2] RunnableWithMessageHistory : 최신 버전 스타일
LangChain 최신 대화 관리 클래스 (≥ 0.2.7) <br>
LLM(예: ChatOpenAI)과 대화 기록(MessageHistory)을 연결해 멀티턴 대화(대화 맥락 유지)를 지원한다.

In [3]:
#  LangChain 최신 메모리 구조 예제
# ConversationChain → RunnableWithMessageHistory로 변경됨

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# 1) LLM 모델 정의
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)

# 2) 세션별 대화 기록 저장소
store = {}

def get_session_history(session_id: str):
    """세션 ID별로 대화 기록을 저장하고 불러오는 함수"""
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

# 3) RunnableWithMessageHistory 구성
conversation = RunnableWithMessageHistory(
    llm,
    get_session_history
)

# 4) 세션 ID 설정
config = {"configurable": {"session_id": "chat-001"}}

# 5) 대화 실행
print("=== 대화 시작 ===")
response1 = conversation.invoke([HumanMessage(content="안녕? 나는 홍길동이야!")], config=config)
print("🤖:", response1.content)

response2 = conversation.invoke([HumanMessage(content="내 이름이 뭐지?")], config=config)
print("🤖:", response2.content)

response3 = conversation.invoke([HumanMessage(content="좋아하는 음식은 뭐야?")], config=config)
print("🤖:", response3.content)

# 6) 현재 메모리(대화 히스토리) 확인
history = get_session_history("chat-001")
print("\n=== 현재 대화 히스토리 ===")
for msg in history.messages:
    role = "🧑 사용자" if msg.type == "human" else "🤖 AI"
    print(f"{role}: {msg.content}")


=== 대화 시작 ===
🤖: 안녕하세요, 홍길동님! 만나서 반갑습니다. 어떻게 도와드릴까요?
🤖: 당신의 이름은 홍길동입니다! 다른 질문이나 도움이 필요하신 부분이 있으면 말씀해 주세요.
🤖: 저는 인공지능이라 음식을 먹거나 좋아할 수는 없지만, 많은 사람들이 좋아하는 음식에 대해 알고 있습니다. 예를 들어, 피자, 초밥, 떡볶이 같은 음식들이 인기가 많죠. 홍길동님은 어떤 음식을 좋아하시나요?

=== 현재 대화 히스토리 ===
🧑 사용자: 안녕? 나는 홍길동이야!
🤖 AI: 안녕하세요, 홍길동님! 만나서 반갑습니다. 어떻게 도와드릴까요?
🧑 사용자: 내 이름이 뭐지?
🤖 AI: 당신의 이름은 홍길동입니다! 다른 질문이나 도움이 필요하신 부분이 있으면 말씀해 주세요.
🧑 사용자: 좋아하는 음식은 뭐야?
🤖 AI: 저는 인공지능이라 음식을 먹거나 좋아할 수는 없지만, 많은 사람들이 좋아하는 음식에 대해 알고 있습니다. 예를 들어, 피자, 초밥, 떡볶이 같은 음식들이 인기가 많죠. 홍길동님은 어떤 음식을 좋아하시나요?


### [3] InMemoryChatMessageHistory
- 대화 기록(Chat Messages)을 메모리(RAM)에 저장하는 기본 클래스
- 한 세션(session_id) 동안 사용자 메시지와 AI 응답을 순차적으로 리스트 형태로 보관
- 프로그램이 종료되면 기록이 사라지는 휘발성(Volatile) 구조
- RunnableWithMessageHistory와 함께 사용해 멀티턴 대화 관리에 활용됨

In [4]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.messages import HumanMessage, AIMessage

# 1) 메모리 객체 생성
history = InMemoryChatMessageHistory()

# 2) 사용자 메시지 추가
history.add_message(HumanMessage(content="안녕?"))

# 3) AI 응답 추가
history.add_message(AIMessage(content="안녕하세요! 만나서 반가워요."))

# 4) 기록 확인
print("=== 대화 기록 ===")
for msg in history.messages:
    role = "🧑 사용자" if msg.type == "human" else "🤖 AI"
    print(f"{role}: {msg.content}")

# 프로그램이 종료되면 기록이 사라지는 휘발성(volatile) 구조

=== 대화 기록 ===
🧑 사용자: 안녕?
🤖 AI: 안녕하세요! 만나서 반가워요.


In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.chat_history import InMemoryChatMessageHistory

# 1) LLM 초기화
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.4)

# 2) 메모리(대화 이력) 생성
history = InMemoryChatMessageHistory()

print("=== InMemoryChatMessageHistory 멀티턴 예제 ===")
print("대화를 입력하세요. (exit 입력 시 종료)\n")

while True:
    user_input = input("🧑 사용자: ")
    if user_input.lower() in ["exit", "quit", "종료"]:
        print("\n대화 종료!")
        break

    # 사용자 메시지 저장
    history.add_message(HumanMessage(content=user_input))

    # 🔹 LLM 호출 시, 이전 대화 이력 전체를 전달
    messages = history.messages  # 지금까지의 모든 대화
    ai_response = llm.invoke(messages)

    # AI 응답 출력 및 기록
    print("🤖 AI:", ai_response.content)
    history.add_message(AIMessage(content=ai_response.content))


=== InMemoryChatMessageHistory 멀티턴 예제 ===
대화를 입력하세요. (exit 입력 시 종료)



🧑 사용자:  안녕 나는 홍길동이야


🤖 AI: 안녕하세요, 홍길동님! 어떻게 도와드릴까요?


🧑 사용자:  내이름이 뭐지


🤖 AI: 당신의 이름은 홍길동입니다. 다른 질문이나 도움이 필요하신 것이 있나요?


🧑 사용자:  exit



대화 종료!


In [6]:
messages

[HumanMessage(content='안녕 나는 홍길동이야', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕하세요, 홍길동님! 어떻게 도와드릴까요?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='내이름이 뭐지', additional_kwargs={}, response_metadata={}),
 AIMessage(content='당신의 이름은 홍길동입니다. 다른 질문이나 도움이 필요하신 것이 있나요?', additional_kwargs={}, response_metadata={})]

### [4] FileChatMessageHistory
: 대화 이력을 로컬 파일(JSON)로 저장하는 클래스

In [7]:
# !pip install langchain-community

In [8]:
from langchain_community.chat_message_histories import FileChatMessageHistory
from langchain_core.messages import HumanMessage, AIMessage

# 1) FileChatMessageHistory 생성
history = FileChatMessageHistory("chat_history.json")

# 2) 메시지 추가
history.add_message(HumanMessage(content="안녕?"))
history.add_message(AIMessage(content="안녕하세요! 만나서 반가워요."))

# 3) 현재 메시지 확인
print("=== 대화 이력 ===")
for msg in history.messages:
    role = "🧑 사용자" if msg.type == "human" else "🤖 AI"
    print(f"{role}: {msg.content}")

# 4) 프로그램 종료 후 다시 실행해도 chat_history.json 파일에 저장된 내용이 유지됨


=== 대화 이력 ===
🧑 사용자: 안녕?
🤖 AI: 안녕하세요! 만나서 반가워요.
🧑 사용자: 안녕?
🤖 AI: 안녕하세요! 만나서 반가워요.


In [9]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_community.chat_message_histories import FileChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# 1) LLM 모델
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.4)

# 2) 세션별 파일 저장소 생성 함수
def get_session_history(session_id: str):
    file_path = f"history_{session_id}.json"
    return FileChatMessageHistory(file_path)

# 3) RunnableWithMessageHistory 구성
conversation = RunnableWithMessageHistory(llm, get_session_history)

# 4) 세션 설정
config = {"configurable": {"session_id": "user-001"}}

# 5) 대화 실행
response1 = conversation.invoke([HumanMessage(content="안녕? 나는 홍길동이야!")], config=config)
print("🤖:", response1.content)

response2 = conversation.invoke([HumanMessage(content="내 이름이 뭐지?")], config=config)
print("🤖:", response2.content)

# 이 코드를 실행하면 history_user-001.json 파일이 자동으로 생성되고,
# 모든 대화 내용이 파일에 순차적으로 누적 저장됩니다.

🤖: 안녕하세요, 홍길동님! 다시 만나서 반갑습니다. 오늘은 어떤 이야기를 나눠볼까요?
🤖: 당신의 이름은 홍길동입니다! 다른 질문이 있으시면 언제든지 말씀해 주세요.


### [5] RedisChatMessageHistory
메모리에 휘발되는 InMemoryChatMessageHistory 대신 Redis DB 서버에 대화 내용을 저장해서 <br>
프로그램을 꺼도, 혹은 여러 사용자가 동시에 접속해도 이전 대화가 유지되는 구조

In [10]:
# !pip install redis

In [11]:
from langchain_community.chat_message_histories import RedisChatMessageHistory
from langchain_core.messages import HumanMessage, AIMessage

# 1) Redis 서버 연결 설정
# host, port, session_id는 자유롭게 변경 가능
history = RedisChatMessageHistory(
    url="redis://localhost:6379/0",  # Redis 서버 주소
    session_id="user-001"            # 세션 구분용 ID
)

# 2) 메시지 추가
history.add_message(HumanMessage(content="안녕?"))
history.add_message(AIMessage(content="안녕하세요! 반가워요"))

# 3) 현재 대화 이력 확인
print("=== Redis 저장된 대화 이력 ===")
for msg in history.messages:
    role = "사용자" if msg.type == "human" else "AI"
    print(f"{role}: {msg.content}")

# Redis 내부에 다음과 같이 저장됨
# KEY: chat_history:user-001
# VALUE: [
#   {"type": "human", "content": "안녕?"},
#   {"type": "AI", "content": "안녕하세요! 반가워요"}
# ]

=== Redis 저장된 대화 이력 ===
사용자: 안녕?
AI: 안녕하세요! 반가워요
사용자: 안녕?
AI: 안녕하세요! 반가워요


In [12]:
# Redis에 저장된 키와 내용 확인하기 
import redis
import json

#  Redis 연결
r = redis.Redis(host='localhost', port=6379, db=0)

#  모든 키 확인
keys = r.keys('*')

if not keys:
    print("❌ 현재 Redis에 저장된 키가 없습니다.")
else:
    print("📦 Redis에 저장된 키와 내용:")
    print("=" * 50)

    for k in keys:
        key = k.decode()
        dtype = r.type(k).decode()

        print(f"🔑 {key}  |  타입: {dtype}")

        if dtype == "list":
            messages = r.lrange(k, 0, -1)
            for i, msg in enumerate(messages, 1):
                try:
                    data = json.loads(msg)
                    msg_type = data.get("type", "")
                    content = data.get("data", {}).get("content", "")
                    # 보기 좋은 출력 포맷
                    print(f"  💬 {i}. {{")
                    print(f'    "type": "{msg_type}",')
                    print(f'    "data": {{"content": "{content}"}}')
                    print("  }")
                except Exception:
                    print(f"  💬 {i}. {msg.decode(errors='ignore')}")
            print("-" * 60)


📦 Redis에 저장된 키와 내용:
🔑 message_store:chat-redis-001  |  타입: list
  💬 1. {
    "type": "ai",
    "data": {"content": "당신의 이름은 홍길동입니다. 다른 질문이나 도움이 필요하신 것이 있으면 말씀해 주세요!"}
  }
  💬 2. {
    "type": "human",
    "data": {"content": "내 이름이 뭐지?"}
  }
  💬 3. {
    "type": "ai",
    "data": {"content": "안녕하세요, 홍길동님! 만나서 반갑습니다. 어떻게 도와드릴까요?"}
  }
  💬 4. {
    "type": "human",
    "data": {"content": "안녕? 나는 홍길동이야!"}
  }
------------------------------------------------------------
🔑 message_store:user-001  |  타입: list
  💬 1. {
    "type": "ai",
    "data": {"content": "안녕하세요! 반가워요"}
  }
  💬 2. {
    "type": "human",
    "data": {"content": "안녕?"}
  }
  💬 3. {
    "type": "ai",
    "data": {"content": "안녕하세요! 반가워요"}
  }
  💬 4. {
    "type": "human",
    "data": {"content": "안녕?"}
  }
------------------------------------------------------------


In [13]:
# Redis에 저장된 내용 삭제하기 
import redis

# Redis 연결
r = redis.Redis(host='localhost', port=6379, db=0)

# 전체 삭제
r.flushall()  # 모든 데이터 삭제

# 현재 DB만 삭제
r.flushdb()   # 기본적으로 db=0이므로 동일

# 특정 세션만 삭제
r.delete("chat_history:user-001")

print("Redis 데이터 삭제 완료!")


Redis 데이터 삭제 완료!


In [14]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_community.chat_message_histories import RedisChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# 1) LLM 모델 정의
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.4)

# 2) Redis 기반 대화 저장소 함수
def get_session_history(session_id: str):
    return RedisChatMessageHistory(
        url="redis://localhost:6379/0",
        session_id=session_id
    )

# 3) RunnableWithMessageHistory 생성
conversation = RunnableWithMessageHistory(llm, get_session_history)

# 4) 세션 설정
config = {"configurable": {"session_id": "chat-redis-001"}}

# 5) 대화 실행
response1 = conversation.invoke([HumanMessage(content="안녕? 나는 홍길동이야!")], config=config)
print("🤖:", response1.content)

response2 = conversation.invoke([HumanMessage(content="내 이름이 뭐지?")], config=config)
print("🤖:", response2.content)

# 6) Redis에 저장된 대화 확인
history = get_session_history("chat-redis-001")
print("\n=== Redis 저장된 대화 이력 ===")
for msg in history.messages:
    role = "🧑 사용자" if msg.type == "human" else "🤖 AI"
    print(f"{role}: {msg.content}")


🤖: 안녕하세요, 홍길동님! 만나서 반갑습니다. 어떻게 도와드릴까요?
🤖: 당신의 이름은 홍길동입니다! 다른 질문이나 궁금한 점이 있으면 말씀해 주세요.

=== Redis 저장된 대화 이력 ===
🧑 사용자: 안녕? 나는 홍길동이야!
🤖 AI: 안녕하세요, 홍길동님! 만나서 반갑습니다. 어떻게 도와드릴까요?
🧑 사용자: 내 이름이 뭐지?
🤖 AI: 당신의 이름은 홍길동입니다! 다른 질문이나 궁금한 점이 있으면 말씀해 주세요.


### [6] SQLChatMessageHistory
: 대화(Message) 기록을 SQL 데이터베이스에 저장하는 LangChain의 메모리 클래스

In [15]:
# !pip install sqlalchemy

In [22]:
from sqlalchemy import create_engine
from langchain_community.chat_message_histories import SQLChatMessageHistory

# 1) DB 연결 엔진 생성
engine = create_engine("sqlite:///chat_history.db")

# 2) SQLChatMessageHistory 객체 생성
history = SQLChatMessageHistory(
    session_id="user-001",
    connection=engine    
)

# 3) 메시지 추가 예시
from langchain_core.messages import HumanMessage, AIMessage
history.add_message(HumanMessage(content="안녕?"))
history.add_message(AIMessage(content="안녕하세요! 반가워요."))

# 4) 저장된 대화 이력 확인
print("=== SQL 저장된 대화 이력 ===")
for msg in history.messages:
    role = "🧑 사용자" if msg.type == "human" else "🤖 AI"
    print(f"{role}: {msg.content}")

=== SQL 저장된 대화 이력 ===
🧑 사용자: 안녕?
🤖 AI: 안녕하세요! 반가워요.


In [17]:
# DB 내용 확인하기
from sqlalchemy import create_engine, text

# 1) SQLite 파일(DB 연결)
engine = create_engine("sqlite:///chat_history.db")

# 2) 저장된 전체 메시지 확인
with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM message_store"))
    rows = result.fetchall()

# 3) 결과 출력
print("=== 저장된 대화 이력 ===")
for row in rows:
    print(row)


=== 저장된 대화 이력 ===
(1, 'chat-sql-001', '{"type": "human", "data": {"content": "\\uc548\\ub155? \\ub098\\ub294 \\ud64d\\uae38\\ub3d9\\uc774\\uc57c!", "additional_kwargs": {}, "response_metadata": {}, "type": "human", "name": null, "id": null}}')
(2, 'chat-sql-001', '{"type": "ai", "data": {"content": "\\uc548\\ub155\\ud558\\uc138\\uc694, \\ud64d\\uae38\\ub3d9\\ub2d8! \\ub9cc\\ub098\\uc11c \\ubc18\\uac11\\uc2b5\\u ... (825 characters truncated) ... output_tokens": 21, "total_tokens": 38, "input_token_details": {"audio": 0, "cache_read": 0}, "output_token_details": {"audio": 0, "reasoning": 0}}}}')
(3, 'chat-sql-001', '{"type": "human", "data": {"content": "\\ub0b4 \\uc774\\ub984\\uc774 \\ubb50\\uc9c0?", "additional_kwargs": {}, "response_metadata": {}, "type": "human", "name": null, "id": null}}')
(4, 'chat-sql-001', '{"type": "ai", "data": {"content": "\\ub2f9\\uc2e0\\uc758 \\uc774\\ub984\\uc740 \\ud64d\\uae38\\ub3d9\\uc785\\ub2c8\\ub2e4! \\ub2e4\\ub978 \\uc9c8\\u ... (898 characters tru

In [18]:
# SQL DB 삭제: 특정 세션 대화만 삭제
from langchain_community.chat_message_histories import SQLChatMessageHistory
from sqlalchemy import create_engine

# 1) DB 연결
engine = create_engine("sqlite:///chat_history.db")

# 2) 세션 ID 지정
session_id = "user-001"

# 3) 해당 세션의 히스토리 불러오기
history = SQLChatMessageHistory(session_id=session_id, connection=engine)

# 4) 대화 전체 삭제
history.clear()  # 이 한 줄로 세션 내 모든 메시지 삭제

print(f"세션 {session_id}의 대화 내용이 모두 삭제되었습니다.")


세션 user-001의 대화 내용이 모두 삭제되었습니다.


In [19]:
# SQL DB 삭제: 테이블내의 모든 데이터 삭제제
from sqlalchemy import create_engine, text

# 1) DB 연결
engine = create_engine("sqlite:///chat_history.db")

# 2) 모든 메시지 삭제 (테이블 전체 비움)
with engine.connect() as conn:
    conn.execute(text("DELETE FROM message_store"))  #  모든 메시지 행 삭제
    conn.commit()

print("DB의 모든 대화 내용이 삭제되었습니다.")


DB의 모든 대화 내용이 삭제되었습니다.


In [20]:
from sqlalchemy import create_engine, text

engine = create_engine("sqlite:///chat_history.db")

with engine.connect() as conn:
    conn.execute(text("DROP TABLE IF EXISTS message_store"))
    conn.commit()

print("message_store 테이블이 완전히 삭제되었습니다.")


message_store 테이블이 완전히 삭제되었습니다.


In [23]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT COUNT(*) FROM message_store"))
    count = result.scalar()
    print("남은 메시지 수:", count)

남은 메시지 수: 2


In [24]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import SQLChatMessageHistory
from sqlalchemy import create_engine

# 1) LLM 정의
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)

# 2) DB 연결 생성
engine = create_engine("sqlite:///chat_history.db")

# 3) 세션별 저장소
def get_session_history(session_id: str):
    return SQLChatMessageHistory(
        session_id=session_id,
        connection=engine   # 신버전 스타일
    )

# 4) RunnableWithMessageHistory 구성
conversation = RunnableWithMessageHistory(llm, get_session_history)
config = {"configurable": {"session_id": "chat-sql-001"}}

# 5) 대화 실행
response1 = conversation.invoke([HumanMessage(content="안녕? 나는 홍길동이야!")], config=config)
print("🤖:", response1.content)

response2 = conversation.invoke([HumanMessage(content="내 이름이 뭐지?")], config=config)
print("🤖:", response2.content)

# 이 코드를 실행하면 chat_history.db 안에 대화가 자동으로 누적 저장된다

🤖: 안녕하세요, 홍길동님! 만나서 반갑습니다. 어떻게 도와드릴까요?
🤖: 당신의 이름은 홍길동입니다! 다른 질문이나 궁금한 점이 있으신가요?


In [25]:
from sqlalchemy import create_engine, text

# 1) SQLite 파일(DB 연결)
engine = create_engine("sqlite:///chat_history.db")

# 2) 저장된 전체 메시지 확인
with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM message_store"))
    rows = result.fetchall()

# 3) 결과 출력
print("=== 저장된 대화 이력 ===")
for row in rows:
    print(row)


=== 저장된 대화 이력 ===
(1, 'user-001', '{"type": "human", "data": {"content": "\\uc548\\ub155?", "additional_kwargs": {}, "response_metadata": {}, "type": "human", "name": null, "id": null}}')
(2, 'user-001', '{"type": "ai", "data": {"content": "\\uc548\\ub155\\ud558\\uc138\\uc694! \\ubc18\\uac00\\uc6cc\\uc694.", "additional_kwargs": {}, "response_metadata": {}, "type": "ai", "name": null, "id": null, "tool_calls": [], "invalid_tool_calls": [], "usage_metadata": null}}')
(3, 'chat-sql-001', '{"type": "human", "data": {"content": "\\uc548\\ub155? \\ub098\\ub294 \\ud64d\\uae38\\ub3d9\\uc774\\uc57c!", "additional_kwargs": {}, "response_metadata": {}, "type": "human", "name": null, "id": null}}')
(4, 'chat-sql-001', '{"type": "ai", "data": {"content": "\\uc548\\ub155\\ud558\\uc138\\uc694, \\ud64d\\uae38\\ub3d9\\ub2d8! \\ub9cc\\ub098\\uc11c \\ubc18\\uac11\\uc2b5\\u ... (825 characters truncated) ... output_tokens": 21, "total_tokens": 38, "input_token_details": {"audio": 0, "cache_read": 0}, "ou

In [26]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT COUNT(*) FROM message_store"))
    count = result.scalar()
    print("남은 메시지 수:", count)

남은 메시지 수: 6
